In [1]:
import os
import json
import boto3
import pandas as pd
import numpy as np
from boto3.dynamodb.types import TypeSerializer, TypeDeserializer
from boto3.dynamodb.conditions import Key
from decimal import Decimal
from dotenv import load_dotenv
from datetime import datetime
import json

load_dotenv()

True

In [2]:

ts= TypeSerializer()
td = TypeDeserializer()

class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return str(obj)
    return json.JSONEncoder.default(self, obj)

In [88]:
def query_sql(beer_id, dynamodb=None):
    if not dynamodb:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))
#    stmt = "SELECT BEER_ID,BEER_NAME,BEER_SRC,BEER_DATE,LABEL_IMG,OG,FG,FERMENT_START,FERMENT_END,OUTPUT,STOCK FROM BREW_LOG"
    stmt = "SELECT BEER_ID,BEER_NAME,BEER_SRC,BEER_DATE,LABEL_IMG,OG,FG,FERMENT_START,FERMENT_END,STOCK,\"OUTPUT\" FROM BREW_LOG WHERE begins_with(BEER_ID,'TEST')=false"
   # print(lambda_handler(event=brew1))
    response = dyna_clnt.execute_statement(Statement=stmt)
    #print("HI")
    return response    
 

In [89]:
lines = query_sql('all')


In [90]:
print(lines)
df_raw = pd.json_normalize(lines, record_path =['Items'])

{'Items': [{'FERMENT_END': {'S': '2020-06-14'}, 'OUTPUT': {'S': '9'}, 'BEER_DATE': {'S': '2020-05-16'}, 'OG': {'S': '1.047'}, 'BEER_ID': {'S': 'Molotov Red Lager'}, 'BEER_NAME': {'S': 'Lamentations Lockdown Lager'}, 'LABEL_IMG': {'S': 'images/lyrb_MRL_2020_05_16.jpg'}, 'FG': {'S': '1.009'}, 'FERMENT_START': {'S': '2020-05-16'}, 'BEER_SRC': {'S': 'http://beerguevara.com/product/molotov-refill-red-lager/'}, 'STOCK': {'S': '0'}}, {'FERMENT_END': {'S': '2021-12-01'}, 'OUTPUT': {'S': '34'}, 'BEER_DATE': {'S': '2021-11-20'}, 'OG': {'S': '1.054'}, 'BEER_ID': {'S': 'BAD_SANTA'}, 'BEER_NAME': {'S': 'Bad Santa Christmas Ale'}, 'LABEL_IMG': {'S': 'images/lyrb_BS_2021_12_01.jpg'}, 'FG': {'S': '1.008'}, 'FERMENT_START': {'S': '2021-11-20'}, 'BEER_SRC': {'S': 'http://beerguevara.com/product/molotov-refill-american-amber-ale/'}, 'STOCK': {'S': '0'}}, {'FERMENT_END': {'S': '2022-12-22'}, 'OUTPUT': {'S': '34'}, 'BEER_DATE': {'S': '2022-12-10'}, 'OG': {'S': '1.042'}, 'BEER_ID': {'S': 'RP_PA'}, 'BEER_NAM

In [77]:
print(df_raw)

   FERMENT_END.S OUTPUT.S BEER_DATE.S   OG.S              BEER_ID.S  \
0     2020-06-14        9  2020-05-16  1.047      Molotov Red Lager   
1     2021-12-01       34  2021-11-20  1.054              BAD_SANTA   
2     2020-12-12        9  1910-12-12  1.047  TESTMolotv Red Lager2   
3     2022-12-22       34  2022-12-10  1.042                  RP_PA   
4     2021-07-22       34  2021-07-03  1.042                  THWAA   
5            NaN      NaN  2021-09-05    NaN           TESTSCWG_IPA   
6     2021-09-15       34  2021-09-05  1.050               SCWG_IPA   
7     2020-07-31        9  2020-07-19  1.044              Amber Ale   
8     2020-10-31        9  2020-10-24  1.052              Amber Ale   
9     2020-08-23        8  2020-08-23  1.048    Molotov Pale Ale v2   
10    2020-10-10        9  2020-09-18  1.044    Molotov Red Lagerv2   
11    2020-08-15        9  2020-07-18  1.047        Dry Irish Stout   
12    2020-05-09        8  2020-05-01  1.058       Molotov Pale Ale   
13    